In [2]:
from heapq import heappop, heappush

def parse_maze(input_str):
    maze = [list(line) for line in input_str.strip().split("\n")]
    start, end = None, None
    for y, row in enumerate(maze):
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)
    return maze, start, end

def solve_maze(input_str):
    # Parse the maze
    maze, start, end = parse_maze(input_str)
    
    # Define movement directions: (dx, dy, direction name)
    directions = [(0, -1, 'N'), (1, 0, 'E'), (0, 1, 'S'), (-1, 0, 'W')]
    direction_map = {d[2]: i for i, d in enumerate(directions)}

    # Priority queue for Dijkstra's
    pq = []
    heappush(pq, (0, start[0], start[1], 'E'))  # (cost, x, y, facing)

    # Visited set: (x, y, facing)
    visited = set()
    cost_map = {}  # To track the minimum cost for each tile

    while pq:
        cost, x, y, facing = heappop(pq)

        # If reached the end, return the cost
        if (x, y) == end:
            return cost, cost_map

        # Skip if already visited
        if (x, y, facing) in visited:
            continue
        visited.add((x, y, facing))

        # Store the minimum cost for the tile
        cost_map[(x, y)] = cost

        # Current direction index
        current_dir_index = direction_map[facing]

        # Explore neighbors
        for i, (dx, dy, new_dir) in enumerate(directions):
            nx, ny = x + dx, y + dy

            # Check boundaries before proceeding
            if not (0 <= nx < len(maze[0]) and 0 <= ny < len(maze)):
                continue

            # If moving forward
            if i == current_dir_index:
                if maze[ny][nx] != '#':  # Valid forward move
                    heappush(pq, (cost + 1, nx, ny, new_dir))

            # If turning (rotating clockwise or counterclockwise)
            else:
                turn_cost = 1000
                heappush(pq, (cost + turn_cost, x, y, new_dir))

    return float('inf'), cost_map  # No solution found

def find_best_path_tiles(input_str):
    # Solve the maze to get the minimum cost and cost map
    maze, start, end = parse_maze(input_str)
    min_cost, cost_map = solve_maze(input_str)

    # Set to store all the tiles part of the best paths
    best_path_tiles = set()

    # Check all the tiles to see if they are part of a best path
    for (x, y), cost in cost_map.items():
        # If this tile is part of the shortest path (cost == min_cost)
        if cost == min_cost:
            best_path_tiles.add((x, y))

            # Check neighbors: the tile can be part of the best path if it leads to
            # a neighboring tile with the correct cost
            for dx, dy, _ in [(0, -1), (1, 0), (0, 1), (-1, 0)]:
                nx, ny = x + dx, y + dy
                if (nx, ny) in cost_map and cost_map[(nx, ny)] == cost - 1:
                    best_path_tiles.add((x, y))
    
    # Return the number of best path tiles
    return len(best_path_tiles)

# Read input from file
with open("input.txt", "r") as file:
    input_str = file.read()

# Solve for the number of tiles on the best path
result = find_best_path_tiles(input_str)
print("Number of tiles part of the best paths:", result)


ValueError: not enough values to unpack (expected 3, got 2)